In [6]:
import numpy as np
from scipy.sparse import spdiags, kron, eye
import time

# Function to generate A and b
def generate_Ab(resolution):
    dm = lambda n: spdiags([-np.ones(n), np.ones(n)], [0, 1], n, n)
    
    Nx = 8 * resolution
    Ny = 8 * resolution
    Nz = 8 * resolution
    
    dx1 = dm(Nx)
    dx2 = kron(eye(Ny), dx1)
    dy1 = dm(Ny)
    dy2 = kron(dy1, eye(Nx))
    dz1 = dm(Nz)
    
    dx3 = kron(eye(Nz), dx2)
    dy3 = kron(eye(Nz), dy2)
    dz3 = kron(dz1, eye(Nx * Ny))
    
    N = Nx * Ny * Nz
    ZM = np.zeros((N, N))
    
    a = np.block([[ZM, dz3.toarray(), dy3.toarray()],
                  [dz3.toarray(), ZM, dx3.toarray()],
                  [dy3.toarray(), dx3.toarray(), ZM]])
    
    A = a @ a.T
    b = np.ones(A.shape[0])
    
    return A, b

# Main code
def gauss_seidel_solver(resolution, maxit=2000, tol=1):
    # Generate A and b
    A, b = generate_Ab(resolution)

    # Initialization
    x = np.zeros(b.shape)
    r = np.zeros((x.shape[0], maxit))

    # Gauss-Seidel iterations
    start_time = time.time()

    for kk in range(maxit):
        x_k = x.copy()
        
        for ii in range(x.shape[0]):
            S1 = A[ii, :ii] @ x[:ii]
            S2 = A[ii, ii+1:] @ x[ii+1:]
            x[ii] = (b[ii] - S1 - S2) / A[ii, ii]
        
        x_k1 = x.copy()
        r[:, kk] = b - A @ x
        
        # Use 2-norm instead of Frobenius norm
        if np.linalg.norm(r[:, kk], 2) < tol:
            print(f"Converged at iteration {kk}")
            break

    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time} seconds")


resolution = 1
gauss_seidel_solver(resolution)


Converged at iteration 1444
Elapsed time: 21.00933337211609 seconds
